# Create Daily Heists

In [1]:
import os, sys, random, json
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.utils import timezone
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST
import emporium.logic.simulation as SM
print([x for x in dir(EM) if x[0]!='_' and len(x)>8])

['ExpeditionLevel', 'ExpeditionType', 'FactoryUpgrades', 'GothicTower', 'LeagueTrial', 'SingletonUpgrades', 'StorageUpgrades', 'ThiefLevel', 'ThroneUpgrades', 'UnlockableItem', 'UnlockableThief']


In [7]:
import engine.models as GM
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.resource as RS
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['GuildExpedition', 'GuildStage', 'ItemInGuild', 'ItemUnlocked', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Daily Heists

In [9]:
# get user and guild

userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)

if not guildMd:
    responseDx = {
        'thiefLs': None,
        'assetLs': None,
        'message': '* A guild must be chosen in the Account page.',
    }

guildMd

<Guild: Guild object (15)>

In [11]:
currDt = timezone.now()
currMonth = currDt.month
currDay = currDt.day
currWeekDay = currDt.weekday()   # 0 monday, 1 tuesday, .. 6 sunday
currDate = f"{currDt.year}-{str(currMonth).zfill(2)}-{str(currDay).zfill(2)}"

print('month:', currMonth, '| day:', currDay, '| week day:', currWeekDay, '| full: ', currDate)

month: 7 | day: 21 | week day: 6 | full:  2024-07-21


### Guild Tower

In [12]:
# check for existing daily stages

checkStages = GM.GuildStage.objects.filter(
    GuildFK=guildMd, Heist='tower', CreateDate=currDate
    ).values()

if checkStages:
    stageDf = PD.DataFrame(checkStages).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
    stageDf = stageDf.drop_duplicates(subset=['StageNo']).sort_values('StageNo')
    stageLs = NT.DataframeToDicts(stageDf)
    PD.DataFrame(stageLs)[:3]

else:
    print('no stages for', currDate)

,id,CreateDate,Heist,ThroneLevel,StageNo,RoomTypes,Background,BackgroundBias,BaseRewards,RoomRewards,Assignments,StageRewards,ObstaclesR1,ObstaclesR2,ObstaclesR3,ObstaclesR4,ObstaclesR5
0,2531,2024-07-21,tower,1,1,"[balanced, None, None, None, None]",nobleman,"[4, None, None, None, None]","{'Gold': 110, 'Gems': 4, 'Wood': 35, 'Stone': ...","[{'xp': 50}, None, None, None, None]","[{'id': 69, 'Name': 'Yondu', 'Class': 'Burglar...","{'gold': 119, 'gems': 5, 'wood': 31}","[{'World': 1, 'Level': 1, 'Name': 'Sorcerer', ...",None,None,None,None
1,2532,2024-07-21,tower,1,2,"[biased agi, None, None, None, None]",warehouse,"[2, None, None, None, None]","{'Gold': 116, 'Gems': 4, 'Wood': 37, 'Stone': ...","[{'xp': 50}, None, None, None, None]","[{'id': 72, 'Name': 'Artax', 'Class': 'Burglar...","{'gold': 108, 'gems': 3, 'wood': 30}","[{'World': 1, 'Level': 1, 'Name': 'Door', 'Tra...",None,None,None,None
2,2533,2024-07-21,tower,1,3,"[balanced, None, None, None, None]",temple,"[3, None, None, None, None]","{'Gold': 121, 'Gems': 4, 'Wood': 39, 'Stone': ...","[None, None, None, None, None]","[None, None, None, None, None]",None,"[{'World': 1, 'Level': 1, 'Name': 'Sewer Grate...",None,None,None,None


In [13]:
# create when daily update is needed

GM.Guild Stage.objects.filter(GuildFK=guildMd, Heist='tower').delete()

rawStages = list(EM.GothicTower.objects.filter(Keep=guildMd.KeepLevel).values())
lastType = ''
lastBackground = ''

for st in rawStages:

    newStage = GM.GuildStage()
    newStage.GuildFK = guildMd
    newStage.Heist = 'tower'
    newStage.StageNo = st['StageNo']
    newStage.CreateDate = currDate
    newStage.RoomTypes = []
    newStage.CompleteRooms = []
    newStage.BackgroundRoomBias = []
    background = ST.StageBackground(lastBackground)
    newStage.Background = background
    lastBackground = background

    # room 1

    roomType = ST.RandomRoomType(lastType)
    lastType = roomType
    obstacles = ST.AssembleRoom(roomType, st['LevelR1'], st['ObstaclesR1'])
    newStage.RoomTypes.append(roomType)
    newStage.CompleteRooms.append(False)
    newStage.BackgroundRoomBias.append(random.randint(0,1))
    newStage.ObstaclesR1 = obstacles

    # room 2

    if st['LevelR2']:
        roomType = ST.RandomRoomType(lastType)
        lastType = roomType
        obstacles = ST.AssembleRoom(roomType, st['LevelR2'], st['ObstaclesR2'])
        newStage.RoomTypes.append(roomType)
        newStage.CompleteRooms.append(False)
        newStage.BackgroundRoomBias.append(random.randint(0,1))
        newStage.ObstaclesR2 = obstacles
    else:
        newStage.RoomTypes.append(None)
        newStage.CompleteRooms.append(None)
        newStage.BackgroundRoomBias.append(None)

    newStage.save()

SyntaxError: invalid syntax (3466708227.py, line 3)

In [ ]:
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd, Heist='tower').values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
stageDf = stageDf.drop_duplicates(subset=['StageNo']).sort_values('StageNo')
stageLs = NT.DataframeToDicts(stageDf)
PD.DataFrame(stageLs)[2:8]

### Development

In [16]:
# reset the data

GM.GuildStage.objects.filter(GuildFK=guildMd).delete()

guildMd.DungeonCheck = None
guildMd.save()

(24, {'engine.GuildStage': 24})

In [15]:
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd).values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
stageDf

""


In [ ]:

trunkNow = timezone.now().replace(microsecond=0)

dayOfWeek = 'monday'
if trunkNow.weekday() == 1:   dayOfWeek = 'tuesday'
if trunkNow.weekday() == 2:   dayOfWeek = 'wednesday'
if trunkNow.weekday() == 3:   dayOfWeek = 'thursday'
if trunkNow.weekday() == 4:   dayOfWeek = 'friday'
if trunkNow.weekday() == 5:   dayOfWeek = 'saturday'
if trunkNow.weekday() == 6:   dayOfWeek = 'sunday'
dayOfWeek

In [ ]:

trialDay = EM.TrialDay.objects.GetOrNone(WeekDay=dayOfWeek)
trialDay.StageType
